# 02 - Continuous stirred tank reactor (CSTR)

**Amazon SageMaker kernel**: conda_pytorch_p36

## Sending insights to the edge

In this notebook we will learn how to take the model we built in the previous notebook and deploy it to an edge device that is running AWS IoT Greengrass to allow process engineers to leverage machine learning model in their daily operations. This could be either by running the process model to predict likely outcomes for a set of process inputs (what-if analysis) or by solving an optimization problem to recommend process inputs for a desired output (prescriptive analytics).

In [ ]:
import boto3
import sagemaker
import time
from sagemaker.utils import name_from_base

# Insert the name of your S3 bucket from CloudFormation outputs
bucket = "" 
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = "us-east-1"
prefix = "models/pytorch"

assert bucket, "ERROR: Insert the name of your S3 bucket before moving on"

### Compile using SageMaker Neo

Neo is a capability of Amazon SageMaker that allows machine learning models to train once and run anywhere in the cloud and at the edge. SageMaker Neo optimizes the machine learning model for inference. The first cell will load the required libraries and will get the Amazon SageMaker session and execution role to be passed to the compilation jobs later.

In [ ]:
import pandas as pd

from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

sm_client = boto3.client('sagemaker')

The next cell contains the configuration parameters for the compilation job. We need to define the framework in which the model is built (PyTorch), its version, and the shape of the input tensor. We also need to define the paths to the S3 buckets where we will store: (1) the compiled model (machine lerarning model optimized for execution with SageMaker Neo), and (2) packaged model (package that contains the model and all the utilities required for it to run on AWS IoT Greengrass).

The following cells contain the instructions to compile and export the process model `cstr-model`, but you can follow the same approach to compile and export the optimization surrogate model `cstr-optimizer`.

In [ ]:
# Configuration
prefix = "models/pytorch"
data_shape = '{"input0":[1,2]}'
key = "workshop" # Use this key to facilitate complete rebuilds of all of the compilation/packaging

# Target definition
target_device = "lambda"
framework = "PYTORCH"
framework_version = "1.7"
model_name = "cstr-model"
model_version = "1.0.0"
compiled_model_path = "s3://{:s}/compiled".format(bucket)
packaged_model_path = "s3://{:s}/packages".format(bucket)
model_s3_uri = ("s3://{:s}/{:s}/" + model_name + ".tar.gz").format(bucket, prefix)

# Greengrass V2 Component Config
component_config = "{\"ComponentName\":\"" + model_name + "-" + target_device + "-component\", \"ComponentVersion\":\"" + model_version + "\"}"

# Specify a name for your edge packaging job.
edge_packaging_job_name = model_name + "NeoPackage" + key

# Specify a name for the model compile... 
compilation_job_name = name_from_base(model_name + "NeoCompile" + key)

print("Model name: " + model_name + " Version: " + model_version)
print("Target Device: " + target_device)
print("Framework: " + framework + " Version: " + framework_version)
print("Model S3 URI: " + model_s3_uri)
print("Component config: " + component_config)
print("Edge package job name: " + edge_packaging_job_name)
print("Compile job name: " + compilation_job_name)

#### Create a PyTorch model object in Amazon SageMaker

The first step will take the model we built in the previous notebook and put it in a `PyTorchModel` object that also indicates the framework, and the SageMaker session that will be used to compile.

In [ ]:
print("Creating PyTorch model from content: " + model_s3_uri + "...")
sagemaker_model = PyTorchModel(
    model_data = model_s3_uri,
    predictor_cls = Predictor,
    framework_version = framework_version,
    role = role,
    sagemaker_session = sess, 
    entry_point = None,
    py_version = "py3",
    env = {"MMS_DEFAULT_RESPONSE_TIMEOUT": "900"},
)

#### Compile model for device

Once the model has been defined, we will run a compilation job for a Lambda target device. This choice allows for portability as the machine learning model can be executed in the cloud (as a Lambda function) or on the edge (as an AWS IoT Greengrass component).

This step can take around 4 minutes.

In [ ]:
print("Compiling model for device: " + target_device + "...")
compiled_model = sagemaker_model.compile(
    target_instance_family = target_device,
    job_name = compilation_job_name,
    input_shape = data_shape,
    role = role,
    framework = framework.lower(),
    framework_version = framework_version,
    output_path = compiled_model_path,
)
compiled_model.name = "compiled-cstr-model-" + target_device + "-" + key
print("")
compiled_model_file = compiled_model_path + "/" + model_name + "-" + target_device + ".tar.gz"
print("Compiled model: " + compiled_model_file)

### (OPTIONAL) Test the SageMaker Neo optimized model prior to deployment

The machine learning model now has been optimized to run as a Lambda. Are we sure that the model is still working as it used to? Let's test it!

For testing we will use the Python [DLR library](https://pypi.org/project/dlr/), which is a compact runtime for deep learning models compiled by AWS SageMaker Neo, TVM, or Treelite.

We will run the following steps:

- Download the compiled model from S3 as a TGZ file

- Unzip the content of the TGZ file in the `local_dirname` directory

- Instantiate a model from the configuration in `local_dirname`

- Evaluate the model for the same configuration we tested in the previous notebook: `F` is 12 [l/h] and `Q_dot` is -10 [kW]

- Compare the results to the ones obtained in the previous notebook. They will be the same

In [ ]:
local_dirname = "dlr_compiled"

print("Pulling compiled model from S3: " + compiled_model_file)
! aws s3 cp {compiled_model_file} .
    
print("Expanding compiled model into our notebook...")
! rm -rf ./{local_dirname} ; mkdir {local_dirname} ; cd {local_dirname} ; tar xzpf ../{model_name}-{target_device}.tar.gz

In [ ]:
import numpy as np
import dlr

model = dlr.DLRModel(local_dirname)

In [ ]:
def evaluate_dlr_model(model, manipulated_vars):
    """
    Return predictions from machine learning model compiled
    with SageMaker Neo that are loaded in Python with the DLR
    library
    """
    X_test = manipulated_vars[["F", "Q_dot"]].values
    X_test[:,0] = (X_test[:,0]-5.0)/95.0
    X_test[:,1] = (X_test[:,1]+5000.0)/5000.0
    X_test = X_test.astype(np.float32)
    Y_test = model.run(X_test)[0]
    Y_test[:,2] = (Y_test[:,2]*25.0)+125.0
    prediction = pd.DataFrame({"F" : manipulated_vars["F"].values.round(2),
                              "Q_dot": manipulated_vars["Q_dot"].values.round(1),
                              "C_a": Y_test[:,0].round(4),
                              "C_b": Y_test[:,1].round(4),
                              "T_K": Y_test[:,2].round(1)})
    
    return prediction

In [ ]:
# Evaluate predictions
manipulated_vars = pd.DataFrame({"F": [12.0], "Q_dot": [-10.0]})

measurements = evaluate_dlr_model(model, manipulated_vars)

print("Measurements from CSTR:")
print("Flow rate: {:.2f} l/h".format(measurements["F"][0]))
print("Concentration of reactant A: {:.4f} mol A/l".format(measurements["C_a"][0]))
print("Concentration of reactant B: {:.4f} mol B/l".format(measurements["C_b"][0]))
print("Temperature in the cooling jacket: {:.2f} C".format(measurements["T_K"][0]))

### Package model to deploy as AWS IoT Greengrass component

This cell will create a Greengrass v2 component. Make sure to use the `GreengrassV2Component` option for the `PresetDeploymentType`. Once this task is completed you should see a brand new component available in AWS IoT Greengrass.

In [ ]:
print("Packaging model for Greengrass Edge (" + target_device + ")...")
print("Greengrass V2 Component Config: " + component_config)
sm_client.create_edge_packaging_job(
    EdgePackagingJobName=edge_packaging_job_name,
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    ModelName=model_name,
    ModelVersion=model_version,
    OutputConfig={
        "S3OutputLocation": packaged_model_path,
        "PresetDeploymentType": "GreengrassV2Component",
        "PresetDeploymentConfig": component_config
        }
    )

print("DONE. GGV2 Component Created: " + component_config)

## Custom components: Using the machine learning model at the edge

Once the model is ready to be deployed to the edge, we need to define the interfaces and modes of operation of the model. We have defined scripts that establish MQTT interfaces for both the prediction and the optimization cases. These will be added to your Greengrass deployment as components in the next section of the workshop but, for now, we will upload these scripts to S3 to they can used in the definition of the Greengrass components

### Prediction

We need to copy 5 files from our SageMaker notebook to our S3 bucket. The script that contains the logic for running the prediction is `cstr_predictor.py`. Feel free to read it to learn how to interact with your ML model at the edge with gRPC.

In [ ]:
# Upload scripts to run machine learning inference (prediction)
artifact_dir = "s3://" + bucket + "/artifacts/CSTRModelPredictor/1.0.0/"

# Copy the files from our notebook to S3
print("Copying files from the notebook to S3")
! aws s3 cp ./predictor/agent_pb2.py {artifact_dir}
! aws s3 cp ./predictor/agent_pb2_grpc.py {artifact_dir}
! aws s3 cp ./predictor/cstr_predictor.py {artifact_dir}
! aws s3 cp ./predictor/launch_predictor.sh {artifact_dir}
! aws s3 cp ./predictor/stop.sh {artifact_dir}

# Note the S3 file names - we'll need them later
print("")
print("S3 Files for custom inference component artifacts:")
print(artifact_dir + "agent_pb2.py")
print(artifact_dir + "agent_pb2_grpc.py")
print(artifact_dir + "cstr_predictor.py")
print(artifact_dir + "launch_predictor.sh")
print(artifact_dir + "stop.sh")

### Optimization

We also need 5 files to run optimization. See that the `cstr_optimizer.py` runs the same `optimal_manipulated_vars` function we designed in the previous notebook.

In [ ]:
# Upload scripts to run optimization with machine learning model
artifact_dir = "s3://" + bucket + "/artifacts/CSTROptimizer/1.0.0/"

# Copy the files from our notebook to S3
print("Copying files from the notebook to S3")
! aws s3 cp ./optimizer/agent_pb2.py {artifact_dir}
! aws s3 cp ./optimizer/agent_pb2_grpc.py {artifact_dir}
! aws s3 cp ./optimizer/cstr_optimizer.py {artifact_dir}
! aws s3 cp ./optimizer/launch_optimizer.sh {artifact_dir}
! aws s3 cp ./optimizer/stop.sh {artifact_dir}

# Note the S3 file names - we'll need them later
print("")
print("S3 Files for custom inference component artifacts:")
print(artifact_dir + "agent_pb2.py")
print(artifact_dir + "agent_pb2_grpc.py")
print(artifact_dir + "cstr_optimizer.py")
print(artifact_dir + "launch_optimizer.sh")
print(artifact_dir + "stop.sh")

We will also note the address of of our IoTCore endpoint because we will need it when configuring our custom Greengrass components.

In [ ]:
# Note our IoTCore endpoint - we'll need this later
print("")
print("IoTCore Endpoint: ")
! aws iot describe-endpoint --endpoint-type iot:Data-ATS | jq ".endpointAddress" | sed 's/"//g'